In [2]:
import pandas as pd

In [3]:
train = pd.read_csv("../data/train.csv")

In [4]:
train["product_name_split"] = train["product_name"].apply(lambda words : words.lower().replace(",", "").replace("&", "").split(" "))

In [5]:
from collections import Counter, defaultdict
counter = defaultdict(Counter)

for e in train.itertuples():
    counter[e.department_id].update([word for word in e.product_name_split if word != ""])
        

In [6]:
for i in range(21):
    mc = counter[i].most_common(10)
    print(f"{i}: {mc}\n")

0: [('cream', 286), ('ice', 272), ('chicken', 249), ('pizza', 197), ('frozen', 186), ('cheese', 174), ('with', 151), ('chocolate', 142), ('organic', 125), ('free', 102)]

1: [('flavor', 15), ('organic', 15), ('sleep', 15), ('natural', 14), ('liquid', 13), ('melatonin', 12), ('hand', 12), ('tablets', 11), ('mix', 11), ('baby', 11)]

2: [('bread', 233), ('whole', 103), ('wheat', 91), ('grain', 68), ('tortillas', 61), ('buns', 61), ('organic', 60), ('free', 51), ('gluten', 43), ('chocolate', 39)]

3: [('organic', 257), ('salad', 51), ('baby', 47), ('red', 44), ('bag', 38), ('potatoes', 34), ('sweet', 32), ('lettuce', 28), ('apple', 28), ('mushrooms', 27)]

4: [('wine', 52), ('beer', 50), ('ale', 48), ('sauvignon', 40), ('california', 32), ('cabernet', 29), ('red', 26), ('chardonnay', 26), ('lager', 25), ('whiskey', 24)]

5: [('sauce', 122), ('rice', 48), ('organic', 47), ('noodles', 40), ('noodle', 34), ('soup', 31), ('thai', 27), ('curry', 26), ('mild', 25), ('miso', 22)]

6: [('tea', 49

## ラベル推測
0. 冷凍製品
1. コンビニ?
2. パンとかケーキとか
3. 野菜とかハーブとか
4. 酒
5. アジア料理・和食
6. コーヒー・お茶
7. ペット食品
8. 地中海系・フランス系
9. 穀物系・シリアル
10. メイクアップ・コスメ・シャンプー
11. 肉
12. 地中海系・イタリアン・調味料
13. シリアル・木の実
14. 缶詰
15. 乳製品
16. 日用品・生活用品
17. 幼児向け食品・製品
18. お菓子・つまみ
19. 惣菜
20. 食品全般

## 考察
- 特徴的な製品がなければ見分けがつき辛い
- 種類だけでなく、売り場のバリエーションも考慮する
- 似た様な売り場は、バリエーションや傾向が違うかも
- 20は食品で決め手がなければ、指定しても良さそう
- 製品名の長さで切っても良さそう
- ベースラインに当て辛い箇所を探しに行く
- 予測値のエントロピーが高いケース

## ベースライン
- サンプル数を揃える
- 全データを使う様にアンサンブルする
- ロジスティック回帰の出力に大して、rightgbm

In [11]:
from random import sample

zero = train.loc[train.department_id == 1,"product_name"].tolist()
sample(zero, 100)

['Baby Healing Ointment',
 'Simple Syrup',
 'Melatonin 1 Mg Peppermint Sublingual Tablets',
 'Wild Bird Food',
 'Classic Margarita Mix',
 'Peanut Butter Pretzels',
 'Giraffes Diapers Size 4 L',
 'Recolte Wine',
 'Pediatric Electrolyte Fruit Flavor Natural & Artificial Flavor',
 'Melatonin NutraSpray Sublingual Orange Flavor',
 'Rapid Relief Creamy Diaper Rash Ointment',
 'Simply Sleep Nighttime Sleep Aid',
 'California Blush Pink Champagne',
 'Healing Baby Diaper Rash Cream',
 'Natural Calm Anti-Stress Drink',
 'Grape Wine, Coteaux Varois en Provence, 2013',
 'Lavender & Chamomile Hand Soap',
 "Children's Grape 24-Hour",
 'Melatonin+ 200mg L-Theanine Dietary Supplement Liquid Softgels',
 'Vanilla Latte Chilled Espresso Beverage',
 'Whispering Angel Rosé',
 "Children's Chestal Homeopathic Medicine",
 'Greek Style Honey Yogurt',
 'Massage & Play 2 in 1 Soothing Touch Massage & Intimate Pleasure Gel',
 'Eucalyptus Ease Bubble Bath',
 'Natural Sleep Aid',
 'Homeopathic Remedy, for Travel F